In [47]:
# file containing multiple genome and read utility functions

from bm_preproc import *
from kmer_index import *
import bisect

class Read:
    '''Class for reads in case we need to ever do anything else with it'''
    
    def __init__(self,seq,quality,length,num=1):
        self.seq=seq
        self.qual=quality
        self.len=length
        self.num=num
        
def readGenome(file):
    '''function to read and return a genome as one string in fasta format'''
    genome =  ''
    f=open(file)
    for line in f:
        if line[0] == ">": #ignoring the name of the genome
            pass
        else:
            genome += line.strip()
    f.close()
    return genome

def readFastQ(file):
    '''function to read multiple reads (as a list of class Read instances) from a fastq file format'''
    reads = []
    f=open(file)
    while f.readline(): # reading in the name
        seq=f.readline().strip().upper() # reading in the sequence
        f.readline() # reading in the + line
        qual=f.readline().strip() #readling in the quality line
        reads.append(Read(seq,qual,len(seq)))
    f.close()
    return reads        
    
def reverseCompliment(seq):
    '''function to return the reverse complement of given sequence'''
    compliment = {'A':'T','T':'A','C':'G','G':'C','N':'N'} # dictionary of base compliments
    revComp = ''
    for c in seq:
        revComp = compliment[c] + revComp
    return revComp

def naive_match(pattern,text,st_aw=False,mismatch=0):
    '''function to return the occurances of pattern in given text, 
    last argument should be passed as True for searching for the rev compliment of the pattern as well'''
    occurances = []
    num_aligns = 0 # counting number of 
    num_char_comp = 0
    len_p = len(pattern)
    len_t = len(text)
    
    rev_same=False
    if st_aw: # calculate the reverse compliment if argument is true
        revP = reverseCompliment(pattern)
        if pattern == revP: # reverse compliment is the same
            rev_same = True 
        
    for i in range(len_t-len_p+1): # iterate over all possible indices
        notMatch = 0
        num_aligns += 1
        
        for j in range(len_p): # for every character in current possible alignment
            num_char_comp += 1
            if pattern[j] != text[i+j]: # if any character not equal, set match False and take a break
                notMatch+=1
                if notMatch > mismatch: # if number of mismtaches are more than what we allow
                    break
        if notMatch <= mismatch: # if all characters are same
            occurances.append(i)

        if st_aw and not rev_same: # if the reverse compliment also needs to be checked
            notMatch = 0
            for j in range(len_p): # for every character in current possible alignment
                if revP[j] != text[i+j]: # if any character not equal, set match False and take a break
                    notMatch+=1
                    if notMatch > mismatch: # if number of mismtaches are more than what we allow
                        break
            if notMatch <= mismatch: # if all characters are same
                occurances.append(i)
            
    return occurances, num_aligns, num_char_comp


def boyer_moore(pattern,text,bm_obj):
    '''function for read alignment using Boyer Moore algorithm, right now does not include 
    reverse complement and looks for exact match without mismatches'''
    
    len_p = len(pattern)
    len_t = len(text)
    num_align = 0
    num_char_comp = 0
    occurances = []
    i = 0 #iterator for alignments
    
    while i < len_t - len_p + 1: #for each alignment
        
        num_align += 1
        match = True
        shift = 1 # if nothing, increment alignment by this much
        
        for j in range(len_p-1,-1,-1): #for each character in the alignment, starting with the end of the pattern
            num_char_comp += 1
            
            if pattern[j] != text[i+j]: #mismatch occurs
                match = False
                shift_bc = bm_obj.bad_character_rule(j,text[i+j]) # calculate shift using the bad character rule
                shift_gs = bm_obj.good_suffix_rule(j) # calculate shift using the good suffix rule
                shift = max(shift_bc,shift_gs,1) # take the maximum that you can shift
                break 
                
        if match: # if it matches
            occurances.append(i)
            shift = bm_obj.match_skip() # shifts if string matches exactly
            
        i += shift
    
    return occurances,num_align,num_char_comp


def pigeon_hole_index(pattern, text, index_t, mismatch = 0):
    '''approximate matching algorithm with number of mismatches using the pigeon hole principle and indexing'''
    
    len_p = len(pattern)
    len_t = len(text)
    divisions = mismatch + 1
    len_k = int(len_p/divisions) # length of the k-mer
    
    occurances = []
    
    if len_p % divisions != 0 or len_k != index_t.k: 
        print('Cannot use the index as the length of the pattern, number of mismatches, and kmer size not compatible')
        print('Returning empty list')
        return occurances
    
    for i in range(0,len_p,len_k): # for every division of the pattern
        kmer = pattern[i:i+len_k]
        hits = index_t.query(kmer) # places where this kmer exists in the text
        print(len(hits))
        
        for h in hits: # for each of the places where this kmer is found, now we have to check if rest of the pattern matches
    
            if h - i in occurances: # if we already have this offset in the occurances list
                continue
                
            num_mismatch = 0
            
            for b in range(i): # for every character before the current kmer in the pattern
                
                if pattern[b] != text[h-i+b]: # h-i+b is the index in the text for this particular pattern char
                    num_mismatch += 1
                    
                    if num_mismatch > mismatch: # if more than number of mismatched allowed, then break
                        break
                        
            if num_mismatch <= mismatch: # now compare the pattern ahead of the current kmer
                
                for a in range(i+len_k,len_p): # for every char after the current kmer
                    
                    if pattern[a] != text[h-i+a]: #h-i+a is the index in the text for this particular pattern char
                        num_mismatch += 1
                        
                        if num_mismatch > mismatch:
                            break
                            
            if num_mismatch <= mismatch: # found a match with allowed mismatches
                occurances.append(int(h-i)) # offset on the text where this string matches            
    
    return occurances # get only unique occurances


def edit_dist_match(pattern,text):
    
    '''function to return the edit distance of the best match between pattern inside text'''
    len_p = len(pattern)
    len_t = len(text)
    
    Matrix = [[0 for i in range(len_t+1)] for j in range(len_p+1)]  # the dynamic programming matrix, initialized as zeros
    for i in range(1,len_p+1): # initializing the first column as the edit distance between an empty string and the pattern
        Matrix[i][0] = i
        
    #print(Matrix)
        
    # fill the test of the matrix
    for p in range(1,len_p+1): # for every row
        
        for t in range(1,len_t+1): # for every column
            
            dist_top = Matrix[p-1][t] + 1 # edit distance from the top + 1 for deletion in the text
            dist_left = Matrix[p][t-1] + 1 # edit distance from the left +1 for deletion in the pattern
            
            if pattern[p-1] == text[t-1]: # if the two current bases are the same (-1 as we are starting from 1 here)
                dist_diag = Matrix[p-1][t-1]
            else:
                dist_diag = Matrix[p-1][t-1] + 1 # one added for substitution
                
            Matrix[p][t] = min(dist_top,dist_left,dist_diag)
            
    #print(Matrix)
    return min(Matrix[len_p]) # the minimum edit distance for the full pattern


def overlap(read_suf,read_pre,min_length):
    
    '''function to return the length of the longest overlap between suffix of read_suf and prefix of read read_pre'''
    if len(read_suf) < min_length or len(read_pre) < min_length: # if any of the reads is less than the min length
        return 0
    
    min_kmer = read_pre[:min_length] # the minimum prefix that needs to be present in read_suf
    start = 0
    while start != -1: # while we can still find min_kmer in read_suf
        
        start = read_suf.find(min_kmer,start)
        if start != -1: # found occurance, check if the rest if the string matches
            if read_pre.startswith(read_suf[start:]): # if read_pre starts with rest of the suffix, then it is true
                return len(read_suf)-start
            else:
                start+=min_length # move past the current match
                
    return 0 # if it comes here, then no more matches and we did not find what we aere looking for   


def calc_overlap_graph(reads,min_length):
    
    '''function to return the edges and their weights of the overlap graph with edges between reads that overlap by atleast min_length'''
    
    edges = [] # list of edges in the graph
    edge_weights = []
    outgoing = {} # track of all outgoing edges for a node
    incoming = {} # track of all incoming edges for a node
    #nodes_with_suffixes = set() # counting number of nodes with an outgoing edge
    
    # preprocess the reads for kmers
    kmers_dict={} # dictionary to store which reads have index kmers
    for i in range(len(reads)): # for each read
        
        incoming[i] = set() # initializing the incoming and outgoing list for this read
        outgoing[i] = set()
        
        for start in range(reads[i].len - min_length + 1): # for all min length kmers in the read
            cur_kmer = reads[i].seq[start:start+min_length]
            
            if cur_kmer in kmers_dict: # if this kmer already in the dictionary
                kmers_dict[cur_kmer].add(i) # add the index of this read in the dictionary
            else:
                kmers_dict[cur_kmer]=set() # create the set and then add the read index
                kmers_dict[cur_kmer].add(i)
                
    # for each read, find the reads that have the same suffix as the prefix of this read
    for i in range(len(reads)): # for each read
        
        kmer_prefix = reads[i].seq[:min_length] # min prefix that needs to be the same
        possibleOverlaps = kmers_dict[kmer_prefix] # all reads that contain the prefix
        
        for r in possibleOverlaps: # check every read in the set
            
            if r != i: # avoiding overlaps with itself
                over_len = overlap(reads[r].seq,reads[i].seq,min_length) # call the overlap function to get the longest overlap
                
                if over_len != 0: # if there is an overlap
                    index = bisect.bisect_left(edge_weights,over_len) # find the insertion point in the sorted weights list
                    edge_weights.insert(index, over_len) # add the weight
                    edges.insert(index,(r,i)) # add the edge
                    outgoing[r].add(i)
                    incoming[i].add(r)
                    #nodes_with_suffixes.add(r)
    
    return edges, edge_weights, incoming, outgoing


def greedy_shortest_sup(reads,min_length=20):
    
    '''greaedy function to return the shortest superstrings containing the reads 
    that have atleast min_length overlap - partial genome assembly'''
    
    edges,weights,incoming,outgoing=calc_overlap_graph(reads,min_length) # calculating the initial overlap graph
    #weights is sorted list, edges is a list according to weights, incoming and outgoing are dictionaries where the key in node index
    
    Left_Nodes = {i:reads[i] for i in range(len(reads))}# list of reads converted to a dictinary for easy addition and removal 
    new_node_index = len(reads) # tracking the current index assigned to new nodes
    removed_nodes = set() # indices of nodes that have to be removed as we have combined these reads, so that we can check edges
    
    while edges: # while there are any edges left that need to be combined
        
        big_over = weights.pop() # the largest weight
        node1,node2 = edges.pop() # index of nodes corresponding to the weights
        
        if node1 in removed_nodes or node2 in removed_nodes: # if any of the two nodes are already removed, go to next iteration
            continue
            
        #print("Edges to connect: %d %d"%(node1,node2))
        
        #creating a new node with the shortest superstring of node1 and node2
        new_node=Read(Left_Nodes[node1].seq+Left_Nodes[node2].seq[big_over:],
                      Left_Nodes[node1].qual+Left_Nodes[node2].qual[big_over:],
                      Left_Nodes[node1].len+Left_Nodes[node2].len-big_over,
                      Left_Nodes[node1].num+Left_Nodes[node2].num)         
        
        new_edges_weight=set() # set for collecting new edges and weight to be added to the graph
        incoming[new_node_index]=set() # initializing the incoming and outgoing for the new node
        outgoing[new_node_index]=set()
        
        # for all incoming edges for node1, recalculating the weight and updating the corresponding dictionaries
        for inc in incoming[node1]: 
            if inc == new_node_index or inc == node1 or inc == node2:# skip the edges we just joined
                continue
                
            #print("Incoming node1: %d"%(inc))
            new_weight=overlap(Left_Nodes[inc].seq,new_node.seq,min_length) # recalculating the overlap between the incoming node and new node
            new_edges_weight.add((inc,new_node_index,new_weight)) # adding the new edge and weight
            incoming[new_node_index].add(inc) # adding the incoming node for the new node
           
            outgoing[inc].remove(node1) # updating the outgoing for the inc node
            outgoing[inc].add(new_node_index)
        
        # for all outgoing edges for node2, recalculating the weight and updating the corresponding dictionaries
        for out in outgoing[node2]: 
            if out == new_node_index or out == node1 or out == node2:# skip the edges we just joined
                continue
                
            #print("Outgoing node2: %d"%(out))
            new_weight=overlap(new_node.seq,Left_Nodes[out].seq,min_length) # recalculating the overlap between the new node and the outgoing node
            new_edges_weight.add((new_node_index,out,new_weight)) # adding the new edge and weight
            outgoing[new_node_index].add(out) # adding the outgoing node for the new node
           
            incoming[out].remove(node2) # updating the incoming for the out node
            incoming[out].add(new_node_index)
        
        # for all outgoing edges for node1, recalculating the weight and updating the corresponding dictionaries
        for out in outgoing[node1]: 
            if out == node2 or out == new_node_index or out == node1: # skip the edges we just joined
                continue
            
            #print("Outgoing node1: %d"%(out))
            new_weight=overlap(new_node.seq,Left_Nodes[out].seq,min_length) # recalculating the overlap between the new node and the outgoing node
            new_edges_weight.add((new_node_index,out,new_weight)) # adding the new edge and weight
            outgoing[new_node_index].add(out) # adding the outgoing node for the new node
           
            incoming[out].remove(node1) # updating the incoming for the out node
            incoming[out].add(new_node_index)
            
        # for all incoming edges for node2, recalculating the weight and updating the corresponding dictionaries
        for inc in incoming[node2]:
            if inc == node1 or inc == new_node_index or inc == node2: # skip the edges we just joined
                continue
            
            #print("Incoming node2: %d"%(inc))
            new_weight=overlap(Left_Nodes[inc].seq,new_node.seq,min_length) # recalculating the overlap between the incoming node and new node
            new_edges_weight.add((inc,new_node_index,new_weight)) # adding the new edge and weight
            incoming[new_node_index].add(inc) # adding the incoming node for the new node
           
            outgoing[inc].remove(node2) # updating the outgoing for the inc node
            outgoing[inc].add(new_node_index)
            
        #Updating the graph with the new node and edges - also deleting whatever is easy i.e. the dictionaries
        
        Left_Nodes[new_node_index]=new_node # add the new node to the graph with the new index
        new_node_index+=1 # increment the index
        
        #adding node1 and node2 as removed and removing the corresponding nodes from the graph
        removed_nodes.add(node1)
        removed_nodes.add(node2)
        del Left_Nodes[node1]
        del Left_Nodes[node2]
        # deleting the incoming and outgoing list for node1 and node2 as we have created them for the new node
        del incoming[node1]
        del outgoing[node1]
        del incoming[node2]
        del outgoing[node2]
        
        # insert the set of new edges and weights into the edges and the weights sorted list of the graph
        for val in new_edges_weight:
            
            new_1,new_2,new_w=val
            index = bisect.bisect_left(weights,new_w)
            weights.insert(index, new_w) # add the weight
            edges.insert(index,(new_1,new_2)) # add the edge
            
        #print(edges,weights)
        #print(incoming)
        #print(outgoing)
        
        #for k in Left_Nodes.keys(): # print the left over nodes
        #    print(k,Left_Nodes[k].seq)
            
    
    return Left_Nodes.values() # returning the left over nodes that could not be concatenated based on this min_distance
        

In [56]:
'''one line tests for the above implemented functions'''
#g = readGenome('/Users/swatijain/Documents/Personal/GenomeSpecialization/Course4-assignments/lambda_virus.fa')
#print(g[:100])

#reads = readFastQ('/Users/swatijain/Documents/Personal/GenomeSpecialization/Course4-assignments/ERR037900_1.first1000.fastq')
#print(len(reads))
#print(reads[0].seq,reads[0].qual,reads[0].len)

#test_seq = 'ATATCGCG'
#print(reverseCompliment(test_seq))

#p = 'CCC'
#ten_as = 'AAAAAAAAAA'
#t = ten_as + 'CCC' + ten_as + 'GGG' + ten_as
#occurrences = naive_match(p, t, True)
#print(occurrences)

#p = 'CTGT'
#ten_as = 'AAAAAAAAAA'
#t = ten_as + 'CTGT' + ten_as + 'CTTT' + ten_as + 'CGGG' + ten_as
#occurrences = naive_match(p, t)
#print(occurrences)

#p = 'word'
#t = 'there would have been a time for such a word'
#occurrences, num_alignments, num_character_comparisons = naive_match(p, t)
#print(occurrences, num_alignments, num_character_comparisons)

#p = 'needle'
#t = 'needle need noodle needle'
#occurrences, num_alignments, num_character_comparisons = naive_match(p, t)
#print(occurrences, num_alignments, num_character_comparisons)

#p = 'word'
#t = 'there would have been a time for such a word'
#lowercase_alphabet = 'abcdefghijklmnopqrstuvwxyz '
#p_bm = BoyerMoore(p, lowercase_alphabet)
#occurrences, num_alignments, num_character_comparisons = boyer_moore(p, t, p_bm)
#print(occurrences, num_alignments, num_character_comparisons)

#p = 'needle'
#t = 'needle need noodle needle'
#p_bm = BoyerMoore(p, lowercase_alphabet)
#occurrences, num_alignments, num_character_comparisons = boyer_moore(p, t, p_bm)
#print(occurrences, num_alignments, num_character_comparisons)


'one line tests for the above implemented functions'

In [3]:
genome = readGenome('/Users/swatijain/Documents/Personal/GenomeSpecialization/Course4-assignments/lambda_virus.fa')
occur = naive_match('AGGAGGTT',genome)
print(occur[0])

49


In [48]:
genome = readGenome('/Users/swatijain/Documents/Personal/GenomeSpecialization/Course4-assignments/chr1.GRCh38.excerpt.fasta')
pattern = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
p_bm = BoyerMoore(pattern,'ACGT')
occurrences, num_alignments, num_character_comparisons = naive_match(pattern,genome)
print(occurrences, num_alignments, num_character_comparisons)
occurrences, num_alignments, num_character_comparisons = boyer_moore(pattern,genome,p_bm)
print(occurrences, num_alignments, num_character_comparisons)

[56922] 799954 984143
[56922] 127974 165191


In [98]:
pattern='GGCGCGGTGGCTCACGCCTGTAAT'
#occur=naive_match(pattern,genome,False,2)
#print(occur)
#len(occur[0])
index_t = Index(genome,8)
#print(index_t.query('GGCGCGGT'))
occur = pigeon_hole_index(pattern,genome,index_t,2)
print(occur)
print(len(occur))

13
17
60
[56922, 84641, 147558, 160729, 191452, 262042, 364263, 657496, 681737, 717706, 160162, 273669, 421221, 429299, 465647, 551134, 724927, 635931, 747359]
19


In [77]:
import bisect
   
class SubseqIndex(object):
    """ Holds a subsequence index for a text T """
    
    def __init__(self, t, k, ival):
        """ Create index from all subsequences consisting of k characters
            spaced ival positions apart.  E.g., SubseqIndex("ATAT", 2, 2)
            extracts ("AA", 0) and ("TT", 1). """
        self.k = k  # num characters per subsequence extracted
        self.ival = ival  # space between them; 1=adjacent, 2=every other, etc
        self.index = []
        self.span = 1 + ival * (k - 1)
        for i in range(len(t) - self.span + 1):  # for each subseq
            self.index.append((t[i:i+self.span:ival], i))  # add (subseq, offset)
        self.index.sort()  # alphabetize by subseq
    
    def query(self, p):
        """ Return index hits for first subseq of p """
        subseq = p[:self.span:self.ival]  # query with first subseq
        i = bisect.bisect_left(self.index, (subseq, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != subseq:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits


In [79]:
index_subseq=SubseqIndex(genome,8,3)
pattern='GGCGCGGTGGCTCACGCCTGTAAT'
hits=index_subseq.query((pattern))
print(len(hits))
hits=index_subseq.query((pattern[1:]))
print(len(hits))
hits=index_subseq.query((pattern[2:]))
print(len(hits))

35
29
15


In [26]:
genome = readGenome('/Users/swatijain/Documents/Personal/GenomeSpecialization/Course4-assignments/chr1.GRCh38.excerpt.fasta')
pattern = 'GATTTACCAGATTGAG'
print(edit_dist_match(pattern,genome))

2


In [33]:
print(overlap('AAABBCDEBBC','BBCDEBBC###',3))

8


In [46]:
allreads=readFastQ('/Users/swatijain/Documents/Personal/GenomeSpecialization/Course4-assignments/test_fastq.fastq')
#for r in allreads:
#    print(r.seq)
#edges,weights,incoming,outgoing=calc_overlap_graph(allreads,4)
#print(edges,weights)
#print(incoming)
#print(outgoing)
assembled=greedy_shortest_sup(allreads,4)
for r in assembled:
    print(r.seq)

Edges to connect: 3 0
Incoming node1: 1
Incoming node1: 2
Outgoing node2: 1
Outgoing node2: 2
Outgoing node2: 4
Outgoing node2: 5
Outgoing node1: 2
Outgoing node1: 4
Incoming node2: 1
[(6, 5), (2, 6), (6, 1), (0, 5), (3, 4), (2, 3), (3, 2), (0, 1), (1, 0), (6, 4), (1, 6), (6, 2), (4, 5), (0, 4), (1, 3), (0, 2), (2, 1)] [4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5]
{1: {2, 6}, 2: {6}, 4: {6}, 5: {4, 6}, 6: {1, 2}}
{1: {6}, 2: {1, 6}, 4: {5}, 5: set(), 6: {1, 2, 4, 5}}
1 TACGTA
2 GTACGT
4 GTACGA
5 TACGAT
6 ACGTACG
Edges to connect: 2 1
Incoming node1: 6
Outgoing node2: 6
Outgoing node1: 6
Incoming node2: 6
[(6, 5), (2, 6), (6, 1), (0, 5), (3, 4), (2, 3), (3, 2), (0, 1), (1, 0), (7, 6), (6, 7), (6, 4), (1, 6), (6, 2), (4, 5), (0, 4), (1, 3), (0, 2)] [4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5]
{4: {6}, 5: {4, 6}, 6: {7}, 7: {6}}
{4: {5}, 5: set(), 6: {4, 5, 7}, 7: {6}}
4 GTACGA
5 TACGAT
6 ACGTACG
7 GTACGTA
Edges to connect: 4 5
Incoming node1: 6
Incoming node2: 6
[(6, 5), 

In [51]:
allreads=readFastQ('/Users/swatijain/Documents/Personal/GenomeSpecialization/Course4-assignments/ERR266411_1.for_asm.fastq')
edges,nodes=calc_overlap_graph(allreads,30)
print(len(edges),len(nodes))

904746 7161


In [54]:
import itertools

def scs(ss):
    """ Returns shortest common superstring of given
        strings, which must be the same length """
    shortest_sup = None
    all_shortest = []
    for ssperm in itertools.permutations(ss):
        sup = ssperm[0]  # superstring starts as first string
        for i in range(len(ss)-1):
            # overlap adjacent strings A and B in the permutation
            olen = overlap(ssperm[i], ssperm[i+1], min_length=1)
            # add non-overlapping portion of B to superstring
            sup += ssperm[i+1][olen:]
        if shortest_sup is None or len(sup) < len(shortest_sup):
            shortest_sup = sup  # found shorter superstring
            all_shortest = [] # reset the list of shortest sequences
            all_shortest.append(sup)
        elif len(sup) == len(shortest_sup): # if the same length
            all_shortest.append(sup)
            
    return all_shortest  # return shortest

In [57]:
#ss=['CCT','CTT','TGC','TGG','GAT','ATT']
strings = ['GAT', 'TAG', 'TCG', 'TGC', 'AAT', 'ATA']
#print(scs(ss))
print(scs(strings))

['TCGATGCAATAG', 'TCGATAGAATGC', 'TCGAATAGATGC', 'TGCAATCGATAG', 'TGCAATAGATCG', 'AATCGATAGTGC', 'AATGCTCGATAG', 'AATAGATCGTGC', 'AATAGATGCTCG', 'AATAGTCGATGC']


In [52]:
allreads=readFastQ('/Users/swatijain/Documents/Personal/GenomeSpecialization/Course4-assignments/ads1_week4_reads.fq')
assembled=greedy_shortest_sup(allreads)
for r in assembled:
    print(r.seq,len(r.seq))
    print(r.seq.count('A'))
    print(r.seq.count('T'))

ACCAAACAAAGTTGGGTAAGGATAGATCAATCAATGATCATATTCTAGTACACTTAGGATTCAAGATCCTATTATCAGGGACAAGAGCAGGATTAGGGATATCCGAGATGGCCACACTTTTGAGGAGCTTAGCATTGTTCAAAAGAAACAAGGACAAACCACCCATTACATCAGGATCCGGTGGAGCCATCAGAGGAATCAAACACATTATTATAGTACCAATTCCTGGAGATTCCTCAATTACCACTCGATCCAGACTACTGGACCGGTTGGTCAGGTTAATTGGAAACCCGGATGTGAGCGGGCCCAAACTAACAGGGGCACTAATAGGTATATTATCCTTATTTGTGGAGTCTCCAGGTCAATTGATTCAGAGGATCACCGATGACCCTGACGTTAGCATCAGGCTGTTAGAGGTTGTTCAGAGTGACCAGTCACAATCTGGCCTTACCTTCGCATCAAGAGGTACCAACATGGAGGATGAGGCGGACCAATACTTTTCACATGATGATCCAAGCAGTAGTGATCAATCCAGGTCCGGATGGTTCGAGAACAAGGAAATCTCAGATATTGAAGTGCAAGACCCTGAGGGATTCAACATGATTCTGGGTACCATTCTAGCCCAGATCTGGGTCTTGCTCGCAAAGGCGGTTACGGCCCCAGACACGGCAGCTGATTCGGAGCTAAGAAGGTGGATAAAGTACACCCAACAAAGAAGGGTAGTTGGTGAATTTAGATTGGAGAGAAAATGGTTGGATGTGGTGAGGAACAGGATTGCCGAGGACCTCTCTTTACGCCGATTCATGGTGGCTCTAATCCTGGATATCAAGAGGACACCCGGGAACAAACCTAGGATTGCTGAAATGATATGTGACATTGATACATATATCGTAGAGGCAGGATTAGCCAGTTTTATCCTGACTATTAAGTTTGGGATAGAAACTATGTATCCTGCTCTTGGACTGCATGAATTTGCTGGTGAGTTATCCACACTTGAGTC